Table 6: Alcohol-specific age-standardised death rates per 100,000 population, deaths registered in Scotland, 2001 to 2016

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/datasets/alcoholspecificdeathsintheukmaindataset')
    sheet = scraper.distribution().as_databaker()

In [2]:
tab = next(tab for tab in sheet if tab.name=='Table 6')

In [3]:
cell = tab.filter('Year')
cell.assert_one()
Sex = cell.shift(0,-1).expand(RIGHT).is_not_whitespace().is_not_blank()
year = cell.expand(DOWN).is_not_whitespace().is_not_blank().is_number()
deaths = cell.shift(1,0).expand(RIGHT).is_not_whitespace().is_not_blank()
observations = deaths.shift(0,1).expand(DOWN).is_not_whitespace().is_not_blank()
Dimensions = [
            HDim(year,'Year',DIRECTLY,LEFT),
            HDim(Sex,'Sex',CLOSEST,LEFT),
            HDim(deaths, 'Alcohol Specific Deaths',DIRECTLY,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','deaths')
            ]
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)
    
new_table = c1.topandas()
import numpy as np
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Year'] = new_table['Year'].apply(lambda x: pd.to_numeric(x, downcast='integer'))

In [4]:
nt1 = new_table[new_table['Alcohol Specific Deaths'] == 'Lower 95% confidence limit']

In [5]:
nt1.columns = ['Lower 95% confidence limit' if x=='Value' else x for x in new_table.columns]

In [6]:
nt2 = new_table[new_table['Alcohol Specific Deaths'] == 'Upper 95% confidence limit']

In [7]:
nt2.columns = ['Upper 95% confidence limit' if x=='Value' else x for x in new_table.columns]

In [8]:
new_table = new_table[new_table['Alcohol Specific Deaths'] != 'Upper 95% confidence limit']

In [9]:
new_table = new_table[new_table['Alcohol Specific Deaths'] != 'Lower 95% confidence limit']

In [10]:
Final_table = pd.merge(new_table, nt1, how = 'inner', on = ['Year','Sex'])

In [11]:
Final_table = Final_table[['Value','Year','Sex','Alcohol Specific Deaths_x','Measure Type_x','Unit_x',
                           'Lower 95% confidence limit']]

In [12]:
Final_table = pd.merge(Final_table, nt2, how = 'inner', on = ['Year','Sex'])

In [13]:
Final_table = Final_table[['Year','Sex','Alcohol Specific Deaths_x','Measure Type','Value','Lower 95% confidence limit', 'Upper 95% confidence limit','Unit']]

In [14]:
Final_table.columns = ['Alcohol Specific Deaths' if x=='Alcohol Specific Deaths_x' else x for x in Final_table.columns]

In [15]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return ''
    else:
        return y
    
Final_table['Lower 95% confidence limit'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Lower 95% confidence limit']), axis = 1)

In [16]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return ''
    else:
        return y
    
Final_table['Upper 95% confidence limit'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Upper 95% confidence limit']), axis = 1)

In [17]:
Final_table['Alcohol Specific Deaths'] = Final_table['Alcohol Specific Deaths'].str.rstrip('1')

In [18]:
Final_table['Sex'] = Final_table['Sex'].map(
    lambda x: {
        'All Persons' : 'T', 
        'Males' : 'M',
        'Females': 'F'         
        }.get(x, x))

In [19]:
Final_table['Age'] =  'All'

In [20]:
Final_table['Geography'] =  'S92000003'

In [21]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return 'Count'
    else:
        return 'Rate per 100,000 persons'
    
Final_table['Measure Type'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Measure Type']), axis = 1)

In [22]:
Final_table

,Year,Sex,Alcohol Specific Deaths,Measure Type,Value,Lower 95% confidence limit,Upper 95% confidence limit,Unit,Age,Geography
0,2001,T,Deaths,Count,1228,,,deaths,All,S92000003
1,2001,T,"Rate per 100,000 people","Rate per 100,000 persons",26,24,27,deaths,All,S92000003
2,2001,F,Deaths,Count,359,,,deaths,All,S92000003
3,2001,F,"Rate per 100,000 females","Rate per 100,000 persons",14,13,16,deaths,All,S92000003
4,2001,M,Deaths,Count,869,,,deaths,All,S92000003
5,2001,M,"Rate per 100,000 males","Rate per 100,000 persons",39,36,41,deaths,All,S92000003
6,2002,T,Deaths,Count,1334,,,deaths,All,S92000003
7,2002,T,"Rate per 100,000 people","Rate per 100,000 persons",28,26,29,deaths,All,S92000003
8,2002,F,Deaths,Count,400,,,deaths,All,S92000003
9,2002,F,"Rate per 100,000 females","Rate per 100,000 persons",16,14,17,deaths,All,S92000003
